CPace sage notebook for generating test vectors for the CPace cipher suites.

The notebook is organized in a series of several subcells.
1.) X25519 definitions and basic string->integer->fieldElement->string conversions back and forth.
2.) Definitions of the Elligator2 primitive straight-line code from the Elligator paper, checked also against the code from the hash_to_curve team.
3.) Test vector generation for X25519 and Elligator2 as ported to sage from the ANSI-C code from the
    Endress+Hauser crypto library.
4.) Implementation and test vector generation for the CPace-specific constructions for generator
    calculation and ISK determination.
5.) Implementation and test vector generation for the AuCPace-specific constructions for generator
    calculation and SK determination.
    
The cells should best be calculated in sequence so that all definitions are available.
Tested with SageMath 9.0 on ubuntu linux.

In [1]:
import sys

from sagelib.CPace_hashing import *


In [2]:
import sys

from sagelib.CPace_string_utils import *


In [3]:
import sys

########## Definitions from RFC 7748 ##################
from sagelib.RFC7748_X448_X25519 import *


In [4]:
def montgomery_get_weak_points(A,field_prime):
    F = GF(field_prime)
    A = F(A)
    num_bits_for_field = ceil(log(float(field_prime),2))
    num_bytes_for_field = floor((num_bits_for_field + 7) / 8)

    nonsquare = get_nonsquare(F)   
    curve = EllipticCurve(F, [0, A , 0, 1 , 0])
    twist = EllipticCurve(F, [0, A * nonsquare, 0, 1 * nonsquare^2, 0])
    
    order_curve = curve.order()
    order_twist = twist.order()
       
    def get_cofactor(order_curve):
        cofactor_candidate = 1;
        while True:
            if order_curve % cofactor_candidate == 0:
                rest = Integer(order_curve / cofactor_candidate)
                if rest.is_prime():
                    return Integer(cofactor_candidate), Integer(order_curve / cofactor_candidate);
            cofactor_candidate += 1

    c,p = get_cofactor(order_curve)
    c_prime,p_prime = get_cofactor(order_twist)
    
    print ("Number of bytes for field: %i" % num_bytes_for_field)
    print ("Number of bits for field: %i" % num_bits_for_field)
    print ("Cofactor curve: %i" % c)
    print ("Cofactor twist: %i" % c_prime)
    print ("Order curve: %i" % (p))
    print ("Order twist: %i" % (p_prime))
    
    weak_points = [];

    for m in range(50):
        u = encodeUCoordinate(m,num_bytes_for_field * 8)

        if is_on_curve(u,A,field_prime):
            u1 = X__(encodeScalar(p,8 * num_bytes_for_field),u,
                     scalar_decoder = decodeUnclampedScalar, A = A, field_prime = field_prime,
                     warnForPointOnTwist = False)
            if not u1 in weak_points:
                weak_points.append(u1)
                print ("Found weak point on curve")
        else:
            u1 = X__(encodeScalar(p_prime,8 * num_bytes_for_field),u,
                     scalar_decoder = decodeUnclampedScalar, A = A, field_prime = field_prime,
                     warnForPointOnTwist = False)
            if not u1 in weak_points:
                weak_points.append(u1)
                print ("Found weak point on twist.")
        if len(weak_points) == (c + c_prime) / 2 - 1: break

    print (weak_points)

    non_canonical_weak_points = []
    for m in weak_points:
        u = decodeUCoordinate(m,num_bits_for_field)
        while True:
            u += field_prime
            if u < 2^(8 * num_bytes_for_field):
                non_canonical_weak_points.append(encodeUCoordinate(u,num_bytes_for_field * 8))
            else:
                break;

    return weak_points,non_canonical_weak_points

if False:
    weak_pts255, nc_weak_pts255 = montgomery_get_weak_points(A_Curve25519,q_Curve25519)
    print (len(weak_pts255))
    print (len(nc_weak_pts255))

    print ("Weak points Curve25519:\n", weak_pts255)
    print ("Weak non-canonical points Curve25519:\n", nc_weak_pts255)

    import sys

    sys.stdout.flush()

    weak_pts448, nc_weak_pts448 = montgomery_get_weak_points(A_Curve448,q_Curve448)

    print ("Weak points Curve448:\n", weak_pts448)
    print ("Weak non-canonical points Curve448:\n", nc_weak_pts448)
else:
    weak_pts255 = [bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'), bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'), bytearray(b'\xec\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x7f'), bytearray(b'\xe0\xebz|;A\xb8\xae\x16V\xe3\xfa\xf1\x9f\xc4j\xda\t\x8d\xeb\x9c2\xb1\xfd\x86b\x05\x16_I\xb8\x00'), bytearray(b'_\x9c\x95\xbc\xa3P\x8c$\xb1\xd0\xb1U\x9c\x83\xef[\x04D\\\xc4X\x1c\x8e\x86\xd8"N\xdd\xd0\x9f\x11W')]
    nc_weak_pts255 = [bytearray(b'\xed\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x7f'), bytearray(b'\xda\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'), bytearray(b'\xee\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x7f'), bytearray(b'\xdb\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'), bytearray(b'\xd9\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'), bytearray(b'\xcd\xebz|;A\xb8\xae\x16V\xe3\xfa\xf1\x9f\xc4j\xda\t\x8d\xeb\x9c2\xb1\xfd\x86b\x05\x16_I\xb8\x80'), bytearray(b'L\x9c\x95\xbc\xa3P\x8c$\xb1\xd0\xb1U\x9c\x83\xef[\x04D\\\xc4X\x1c\x8e\x86\xd8"N\xdd\xd0\x9f\x11\xd7')]

    weak_pts448 =[bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'), bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'), bytearray(b'\xe4\x8b\xba\x0c\xab\xec\x83\x86\\N\xb1k|\x8e\xfd\xf9\xe4\x97\xec\xa9\x0c\x9e\xb0\xf2\xa5\xb1L\xf1t\xf1\xd9R')]
    nc_weak_pts448=[bytearray(b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'), bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff')]


In [5]:
input_scalar = 599189175373896402783756016145213256157230856085026129926891459468622403380588640249457727683869421921443004045221642549886377526240828
input_coor =   382239910814107330116229961234899377031416365240571325148346555922438025162094455820962429142971339584360034337310079791515452463053830
output_coor =  0xce3e4ff95a60dc6697da1db1d85e6afbdf79b50a2412d7546d5f239fe14fbaadeb445fc66a01b0779d98223961111e21766282f73dd96b6f

print (X448(IntegerToByteArray(input_scalar,57),IntegerToByteArray(input_coor,57)))

input_scalar = 31029842492115040904895560451863089656472772604678260265531221036453811406496
input_coor =   34426434033919594451155107781188821651316167215306631574996226621102155684838
output_coor = 0xc3da55379de9c6908e94ea4df28d084f32eccf03491c71f754b4075577a28552

X25519(IntegerToByteArray(input_scalar),IntegerToByteArray(input_coor))

b'\xce>O\xf9Z`\xdcf\x97\xda\x1d\xb1\xd8^j\xfb\xdfy\xb5\n$\x12\xd7Tm_#\x9f\xe1O\xba\xad\xebD_\xc6j\x01\xb0w\x9d\x98"9a\x11\x1e!vb\x82\xf7=\xd9ko'


b'\xc3\xdaU7\x9d\xe9\xc6\x90\x8e\x94\xeaM\xf2\x8d\x08O2\xec\xcf\x03I\x1cq\xf7T\xb4\x07Uw\xa2\x85R'

In [6]:
# 2.) Definitions for the X25519 test cases

class X25519_testCase:
    def __init__(self,u_in, s_in, u_out):
        self.u_in = u_in
        self.s_in = s_in
        self.u_out = u_out

    def runTest(self):
        us = IntegerToByteArray(self.u_in)
        ss = IntegerToByteArray(self.s_in)
        r  = encodeUCoordinate(self.u_out,256)
        u = X25519(ss,us, unclamped_basepoint = True)
        if (u != r):
            print ("Fail")
            print ("Input u :\n0x%032x\n" % self.u_in)
            print ("Input s :\n0x%032x\n" % self.s_in)
            print ("Correct Result :\n0x%032x\n" % self.u_out)
            print ("Actual Result :\n0x%032x\n" % decodeLittleEndian(u,256))
            return False
        print ("Pass")
        return True
    
    def docOutput(self):
        print ("Test case for X25519:")
        print ("u:"),
        print (IntegerToLEPrintString(self.u_in))
        print ("s:"),
        print (IntegerToLEPrintString(self.s_in))
        print ("r:"),
        print (IntegerToLEPrintString(self.u_out))
        

testCases = []

tv = \
    X25519_testCase(0x4c1cabd0a603a9103b35b326ec2466727c5fb124a4c19435db3030586768dbe6,\
                    0xc49a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346a5,\
                    0x5285a2775507b454f7711c4903cfec324f088df24dea948e90c6e99d3755dac3)
testCases.append(tv)


tv = X25519_testCase(0x13a415c749d54cfc3e3cc06f10e7db312cae38059d95b7f4d3116878120f21e5,\
                     0xdba18799e16a42cd401eae021641bc1f56a7d959126d25a3c67b4d1d4e9664b,\
                    0x5779ac7a64f7f8e652a19f79685a598bf873b8b45ce4ad7a7d90e87694decb95)
testCases.append(tv)

tv = X25519_testCase(0,\
                     0xc49a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346a5,\
                     0)
testCases.append(tv)
    
weakp = []
weakp.append(0)
weakp.append(1)
weakp.append(325606250916557431795983626356110631294008115727848805560023387167927233504) #(which has order 8)
weakp.append(39382357235489614581723060781553021112529911719440698176882885853963445705823) #(which also has order 8)
weakp.append(2^255 - 19 - 1)
weakp.append(2^255 - 19)
weakp.append(2^255 - 19 + 1)
weakp.append(2^255 - 19 + 325606250916557431795983626356110631294008115727848805560023387167927233504)
weakp.append(2^255 - 19 + 39382357235489614581723060781553021112529911719440698176882885853963445705823)
weakp.append(2 * (2^255 - 19) - 1)
weakp.append(2 * (2^255 - 19))
weakp.append(2 * (2^255 - 19) + 1)

s_in = 0xff9a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346af;
for x in weakp:
    tv = X25519_testCase (x,s_in,0)
    testCases.append(tv)


for x in testCases:
    x.runTest()

for x in testCases:
    x.docOutput()


Pass
Input point is on the twist! 
Pass
Pass
Pass
Pass
Pass
Pass
Input point is on the twist! 
Pass
Pass
Pass
Pass
Pass
Input point is on the twist! 
Pass
Pass
Pass
Test case for X25519:
u:
e6db6867583030db3594c1a424b15f7c726624ec26b3353b10a903a6d0ab1c4c
s:
a546e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a2244ba449ac4
r:
c3da55379de9c6908e94ea4df28d084f32eccf03491c71f754b4075577a28552
Test case for X25519:
u:
e5210f12786811d3f4b7959d0538ae2c31dbe7106fc03c3efc4cd549c715a413
s:
4b66e9d4d1b4673c5ad22691957d6af5c11b6421e0ea01d42ca4169e7918ba0d
r:
95cbde9476e8907d7aade45cb4b873f88b595a68799fa152e6f8f7647aac7957
Test case for X25519:
u:
0000000000000000000000000000000000000000000000000000000000000000
s:
a546e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a2244ba449ac4
r:
0000000000000000000000000000000000000000000000000000000000000000
Test case for X25519:
u:
0000000000000000000000000000000000000000000000000000000000000000
s:
af46e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a2244ba449aff

In [7]:
def output_test_vectors_for_weak_points_255(file = sys.stdout):
    print ("\n## Test vectors for X25519 low order points\n",file = file)
    print ("Points that need to return neutral element when input to", file = file)
    print ("plain X25519 that also accept un-normalized inputs with", file = file)
    print ("bit #255 set in the input point encoding.", file = file)
    print ("\n~~~", file = file)

    s_in = 0xff9a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346af;

    weak_pts255 = [(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'), 
                   (b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'), 
                   (b'\xec\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x7f'), 
                   (b'\xe0\xebz|;A\xb8\xae\x16V\xe3\xfa\xf1\x9f\xc4j\xda\t\x8d\xeb\x9c2\xb1\xfd\x86b\x05\x16_I\xb8\x00'), 
                   (b'_\x9c\x95\xbc\xa3P\x8c$\xb1\xd0\xb1U\x9c\x83\xef[\x04D\\\xc4X\x1c\x8e\x86\xd8"N\xdd\xd0\x9f\x11W')]
    nc_weak_pts255 = [(b'\xed\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x7f'), 
                      (b'\xda\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'),
                      (b'\xee\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x7f'),
                      (b'\xdb\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'),
                      (b'\xd9\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'),
                      (b'\xcd\xebz|;A\xb8\xae\x16V\xe3\xfa\xf1\x9f\xc4j\xda\t\x8d\xeb\x9c2\xb1\xfd\x86b\x05\x16_I\xb8\x80'), 
                      (b'L\x9c\x95\xbc\xa3P\x8c$\xb1\xd0\xb1U\x9c\x83\xef[\x04D\\\xc4X\x1c\x8e\x86\xd8"N\xdd\xd0\x9f\x11\xd7')]

    weakp = []
    for wp in weak_pts255:
        weakp.append(decodeUCoordinate(wp,255))
    for wp in nc_weak_pts255:
        weakp.append(decodeUCoordinate(wp,256))

    ctr=0;
    for x in weakp:
        print ("u"+'{:01x}'.format(ctr)+":",IntegerToLEPrintString(x), file = file);
        ctr += 1;
    
    print ("\nResults for X25519 implementations not clearing bit #255:", file = file)
    print ("(i.e. with X25519 not implemented according to RFC7748!):", file = file)
    print ("s =", IntegerToLEPrintString(s_in), file = file);
    print ("rN = X25519(s,uX);", file = file)
    ctr=0;
    for x in weakp:
        r = X25519(encodeScalar(s_in,256), encodeUCoordinate(x,256),warnForPointOnTwist=0,unclamped_basepoint = True);
        r = decodeLittleEndian(r,256)
        print ("r"+'{:01x}'.format(ctr)+":",IntegerToLEPrintString(r), file = file);
        ctr += 1;

    print ("\nResults for X25519 implementations that clear bit #255:", file = file)
    print ("(i.e. implemented according to RFC7748!):", file = file)
    print ("s =", IntegerToLEPrintString(s_in), file = file);
    print ("qN = X25519(s, uX & ((1 << 255) - 1));", file = file)
    ctr=0;
    for x in weakp:
        q = X25519(encodeScalar(s_in,256), encodeUCoordinate(x,256),warnForPointOnTwist=0);
        q = decodeLittleEndian(q,256)
        print ("q"+'{:01x}'.format(ctr)+":",IntegerToLEPrintString(q), file = file);
        ctr += 1;
        
    print ("~~~\n", file = file)
output_test_vectors_for_weak_points_255()


## Test vectors for X25519 low order points

Points that need to return neutral element when input to
plain X25519 that also accept un-normalized inputs with
bit #255 set in the input point encoding.

~~~
u0: 0000000000000000000000000000000000000000000000000000000000000000
u1: 0100000000000000000000000000000000000000000000000000000000000000
u2: ecffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff7f
u3: e0eb7a7c3b41b8ae1656e3faf19fc46ada098deb9c32b1fd866205165f49b800
u4: 5f9c95bca3508c24b1d0b1559c83ef5b04445cc4581c8e86d8224eddd09f1157
u5: edffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff7f
u6: daffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
u7: eeffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff7f
u8: dbffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
u9: d9ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
ua: cdeb7a7c3b41b8ae1656e3faf19fc46ada098deb9c32b1fd866205165f49b880
ub: 4c9c95bca3508c24b1d0b1559c83ef5

In [8]:

def output_test_vectors_for_weak_points_448(file = sys.stdout):
    print ("\n## Test vectors for X448 low order points\n",file = file)
    print ("Points that need to return neutral element when input to", file = file)
    print ("plain X448 that also accept non-canonical inputs larger", file = file)
    print ("than the field prime.", file = file)

    s = b""
    while True:
        s = H_SHAKE256().hash(s,56)
        if (s[0] & 3 == 3):
            break;
    
    weak_pts448 = [bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'), bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'), bytearray(b'\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff')]
    nc_weak_pts448= [bytearray(b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'), bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff')]
           
    weakp = []
    for wp in weak_pts448:
        weakp.append(wp)
    for wp in nc_weak_pts448:
        weakp.append(wp)

    ctr=0;
    print ("\n### Weak points for X448 smaller than the field prime (canonical)\n",file = file)
    print ("~~~", file = file)
    for x in weak_pts448:
        tv_output_byte_array(x, 
                         test_vector_name = 'u%i' % ctr, 
                         line_prefix = "  ", max_len = 60, file = file);
        ctr += 1;
    print ("~~~", file = file)
    print ("\n### Weak points for X448 larger or equal to the field prime (non-canonical)\n",file = file)
    print ("~~~", file = file)
    for x in nc_weak_pts448:
        tv_output_byte_array(x, 
                         test_vector_name = 'u%i' % ctr, 
                         line_prefix = "  ", max_len = 60, file = file);
        ctr += 1;
    print ("~~~", file = file)
    
    print ("\n### Expected results for X448\n",file = file)
    ctr=0;
    print ("~~~", file = file)
    tv_output_byte_array(s, 
                         test_vector_name = 'scalar s', 
                         line_prefix = "  ", max_len = 60, file = file);
    for x in weak_pts448:
        res = X448(s,x,warnForPointOnTwist = False)
        res = decodeUCoordinate(res,448)
        res = IntegerToByteArray(res,56)
        tv_output_byte_array(res, 
                         test_vector_name = 'X448(s,u%i)' % ctr, 
                         line_prefix = "  ", max_len = 60, file = file);
        ctr += 1;

    for x in nc_weak_pts448:
        res = X448(s,x,warnForPointOnTwist = False)
        res = decodeUCoordinate(res,448)
        res = IntegerToByteArray(res,56)
        tv_output_byte_array(res, 
                         test_vector_name = 'X448(s,u%i)' % ctr, 
                         line_prefix = "  ", max_len = 60, file = file);
        ctr += 1;
    print ("~~~\n", file = file)
    
    
output_test_vectors_for_weak_points_448()


## Test vectors for X448 low order points

Points that need to return neutral element when input to
plain X448 that also accept non-canonical inputs larger
than the field prime.

### Weak points for X448 smaller than the field prime (canonical)

~~~
  u0: (length: 56 bytes)
    0000000000000000000000000000000000000000000000000000000000
    000000000000000000000000000000000000000000000000000000
  u1: (length: 56 bytes)
    0100000000000000000000000000000000000000000000000000000000
    000000000000000000000000000000000000000000000000000000
  u2: (length: 56 bytes)
    fefffffffffffffffffffffffffffffffffffffffffffffffffffffffe
    ffffffffffffffffffffffffffffffffffffffffffffffffffffff
~~~

### Weak points for X448 larger or equal to the field prime (non-canonical)

~~~
  u3: (length: 56 bytes)
    fffffffffffffffffffffffffffffffffffffffffffffffffffffffffe
    ffffffffffffffffffffffffffffffffffffffffffffffffffffff
  u4: (length: 56 bytes)
    0000000000000000000000000000000000000000000000

In [9]:
    
generate_testvectors_string_functions()


## Definition and test vectors for string utility functions


### prepend_length function


~~~
  def prepend_length_to_bytes(data):
      length_as_utf8_string = chr(len(data)).encode('utf-8')
      return (length_as_utf8_string + data)
~~~


### prepend_length test vectors

~~~
  prepend_length_to_bytes(b""): (length: 1 bytes)
    00
  prepend_length_to_bytes(b"1234"): (length: 5 bytes)
    0431323334
  prepend_length_to_bytes(bytes(range(127))): (length: 128 bytes)
    7f000102030405060708090a0b0c0d0e0f101112131415161718191a1b
    1c1d1e1f202122232425262728292a2b2c2d2e2f303132333435363738
    393a3b3c3d3e3f404142434445464748494a4b4c4d4e4f505152535455
    565758595a5b5c5d5e5f606162636465666768696a6b6c6d6e6f707172
    737475767778797a7b7c7d7e
  prepend_length_to_bytes(bytes(range(128))): (length: 130 bytes)
    c280000102030405060708090a0b0c0d0e0f101112131415161718191a
    1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637
    38393a3b3c3d3e3f404142434445464748494a4b4c4d4e4f5

In [10]:
class G_Montgomery:
    """Here we have common definitions for the X448 and X25519"""
    def sample_scalar(self, deterministic_scalar_for_test_vectors = "False"):
        if deterministic_scalar_for_test_vectors == "False":
            return random_bytes(self.field_size_bytes)
        else:
            H = H_SHA512()
            value = H.hash(deterministic_scalar_for_test_vectors)
            return value[0:self.field_size_bytes]

    def decodeLittleEndian(self, b):
        bits = self.field_size_bits
        num_bytes = floor((bits+7)/8)
        return sum([b[i] << 8*i for i in range(num_bytes)])

    def decodeUCoordinate(self, u):        
        u_list = [b for b in u]
        # Ignore any unused bits.
        if self.field_size_bits % 8:
            u_list[-1] &= (1<<(self.field_size_bits%8))-1
        return self.decodeLittleEndian(u_list)

    def encodeUCoordinate(self,u):
        u = u % self.q
        return IntegerToByteArray(u,self.field_size_bytes)
    
    def find_z_ell2(self,F):
        """ Argument: F, a field object, e.g., F = GF(2^255 - 19) """
        ctr = F.gen()
        while True:
            for Z_cand in (F(ctr), F(-ctr)):
                # Z must be a non-square in F.
                if is_square(Z_cand):
                    continue
                return Z_cand
            ctr += 1
        
    def elligator2(self,r):
        q = self.q
        Fq = GF(q)
        A = Fq(self.A)
        B = Fq(1)
    
        # calculate the appropriate non-square as specified in the hash2curve draft.
        u = Fq(self.find_z_ell2(Fq))
        powerForChi = floor((q-1)/2)
    
        v = - A / (1 + u * r^2)
        epsilon = (v^3 + A * v^2 + B * v)^powerForChi
        x = epsilon * v - (1 - epsilon) * A/2
        return self.encodeUCoordinate(Integer(x))

    def calculate_generator(self, H, PRS, CI, sid, print_test_vector_info = False, file = sys.stdout):
        (gen_string, len_zpad) = generator_string(PRS, G.DSI,CI,sid,H.s_in_bytes)
        string_hash = H.hash(gen_string, self.field_size_bytes)
        u = self.decodeUCoordinate(string_hash)
        result = self.elligator2(u)
        if print_test_vector_info:
            print ("\n###  Test vectors for calculate_generator with group "+self.name+"\n",file=file)
            print ("~~~", file=file)

            print ("  Inputs", file=file)
            print ("    H   =", H.name, "with input block size", H.s_in_bytes, "bytes.", file=file)
            print ("    PRS =", PRS, "; ZPAD length:", len_zpad,"; DSI =", self.DSI, file=file)
            print ("    CI =", CI, file=file)
            print ("    CI =", ByteArrayToLEPrintString(CI), file=file)
            print ("    sid =", ByteArrayToLEPrintString(sid), file=file)
            print ("  Outputs",file=file)
            tv_output_byte_array(string_hash, test_vector_name = "hash generator string", 
                                 line_prefix = "    ", max_len = 60, file=file)
            tv_output_byte_array(IntegerToByteArray(u), test_vector_name = "after decoding to coordinate", 
                                 line_prefix = "    ", max_len = 60, file=file)
            tv_output_byte_array(result, test_vector_name = "generator g", 
                                 line_prefix = "    ", max_len = 60, file=file)
            print ("~~~", file=file)
        return result

    
class G_X25519(G_Montgomery):
    def __init__(self):
        self.I = zero_bytes(32)
        self.field_size_bytes = 32
        self.field_size_bits = 255
        self.DSI = b"CPace255"
        self.DSI_ISK = b"CPace255_ISK"
        self.name = "X25519" # group name
        self.encoding_of_scalar = "little endian"
        
        # curve definitions
        self.q = 2^255 - 19
        self.A = 486662
        
    def scalar_mult(self,scalar,point):
        return X25519(scalar,point)

    def scalar_mult_vfy(self,scalar,point):
        return X25519(scalar,point)

    
class G_X448(G_Montgomery):
    def __init__(self):
        self.I = zero_bytes(56)
        self.field_size_bytes = 56
        self.field_size_bits = 448
        self.DSI = b"CPace448"
        self.DSI_ISK = b"CPace448_ISK"
        self.name = "X448" # group name
        self.encoding_of_scalar = "little endian"
        
        # curve definitions
        self.q = 2^448 - 2^224 - 1
        self.A = 156326
        
    def scalar_mult(self,scalar,point):
        return X448(scalar,point) # yet no definition for X448 in this script file

    def scalar_mult_vfy(self,scalar,point):
        return X448(scalar,point) # yet no definition for X448 in this script file


In [11]:
# 4.) Definitions of the Elligator2 test cases
#
# Elligator 2 test cases
#
#
# Testvector from the NaCl M0 testsuite from E+H
#
G = G_X25519()

etc1_in =  0x00c84eddfa9bcd7973d6021153cd965a8a2fd749135834eaaeb093d2469a14bc
etc1_out = 0x67d305efdb0a7c7f24ce1655ecc103126004ff23d32bfc032428cd75758fb666

etc2_in =  0x7563f23b0c0aa7bc27b2961a4711ba842ba303c57a9534164bf8d3b5d455cf89
etc2_out = 0x08a3bb40e5b594b192d0ee87b663d24e1bc76d2d41c9031962a7ec6cc863b11d

ourResult1 = decodeUCoordinate(G.elligator2(etc1_in),256)
ourResult2 = decodeUCoordinate(G.elligator2(etc2_in),256)

if (ourResult1 != etc1_out):
    print ("Elligator test case #1 failed.")
else:
    print ("Elligator test case #1 pass.")

if (ourResult2 != etc2_out):
    print ("Elligator test case #2 failed.")
else:
    print ("Elligator test case #2 pass.")
    
print ("Elligator test case #1:")
print ("In:  0x%x" % etc1_in)
print ("Out: 0x%x" % etc1_out)

print ("Elligator test case #1:")
print ("In:  0x%x" % etc2_in)
print ("Out: 0x%x" % etc2_out)


Elligator test case #1 pass.
Elligator test case #2 pass.
Elligator test case #1:
In:  0xc84eddfa9bcd7973d6021153cd965a8a2fd749135834eaaeb093d2469a14bc
Out: 0x67d305efdb0a7c7f24ce1655ecc103126004ff23d32bfc032428cd75758fb666
Elligator test case #1:
In:  0x7563f23b0c0aa7bc27b2961a4711ba842ba303c57a9534164bf8d3b5d455cf89
Out: 0x8a3bb40e5b594b192d0ee87b663d24e1bc76d2d41c9031962a7ec6cc863b11d


In [13]:
def CPace_ISK(H, DSI,sid,K,MSGa,MSGb,doPrint = 1, symmetric_execution = False, file = sys.stdout):
    if symmetric_execution:
        concatenated_msg_transcript = oCAT(MSGa,MSGb)
        if doPrint:
            print ("\n###  Test vector for ISK calculation parallel execution\n", file=file)
            print ("~~~", file=file)
        tv_output_byte_array(concatenated_msg_transcript, test_vector_name = "ordered cat of transcript ", 
                         line_prefix = "    ", max_len = 60, file=file)
    else:
        concatenated_msg_transcript = MSGa + MSGb
        if doPrint:
            print ("\n###  Test vector for ISK calculation initiator/responder\n", file=file)
            print ("~~~", file=file)
        tv_output_byte_array(concatenated_msg_transcript, test_vector_name = "unordered cat of transcript ", 
                         line_prefix = "    ", max_len = 60, file=file)
        
    string = prefix_free_cat(DSI,sid,K)+ concatenated_msg_transcript
    ISK = H.hash(string)
    if doPrint:
        tv_output_byte_array(string, test_vector_name = "input to final ISK hash", 
                         line_prefix = "    ", max_len = 60, file=file)
        tv_output_byte_array(ISK, test_vector_name = "ISK result", 
                         line_prefix = "    ", max_len = 60, file=file)
        print ("~~~", file=file)

    return ISK

In [14]:

def generate_test_vector(H,G, with_ANSI_C_initializers = True,file=sys.stdout):
    print ("##  Test vector for CPace using group " + G.name + " and hash "+H.name +"\n", file=file)

    sid = H.hash(b"sid")
    sid = sid [:16]

    PRS = b"password"
    CI = (prepend_length_to_bytes(b"Ainitiator") 
          + prepend_length_to_bytes(b"Bresponder"))

    ADa = b"ADa"
    ADb = b"ADb"

    g = G.calculate_generator(H,PRS,CI,sid, True, file = file)
    
    seed = b""
    while True:
        ya = G.sample_scalar(b"A"+seed)
        Ya = G.scalar_mult(ya, g)
        yb = G.sample_scalar(b"B"+seed)
        Yb = G.scalar_mult(yb, g)
        if not (oCAT(Ya,Yb) == Ya + Yb):
            break;
        seed += b" "
            
    print (type(Ya),Ya)
    print (type(ADa),ADa)
    
    MSGa = prefix_free_cat(Ya,ADa)
    MSGb = prefix_free_cat(Yb,ADb)
   
    print ("\n###  Test vector for MSGa\n", file=file)
    print ("~~~", file=file)
    print ("  Inputs", file=file)
    print ("    ADa =",ADa, file=file)
    tv_output_byte_array(ya, test_vector_name = "ya (" + G.encoding_of_scalar +")", 
                         line_prefix = "    ", max_len = 60, file=file)
    
    print ("  Outputs",file=file)
    tv_output_byte_array(Ya, test_vector_name = "Ya", 
                         line_prefix = "    ", max_len = 60, file=file)
    tv_output_byte_array(MSGa, test_vector_name = "MSGa", 
                         line_prefix = "    ", max_len = 60, file=file)
    print ("~~~", file=file)
    print ("\n###  Test vector for MSGb\n", file=file)
    print ("~~~", file=file)
    print ("  Inputs", file=file)
    print ("    ADb =", ADb, file=file)
    tv_output_byte_array(yb, test_vector_name = "yb (" + G.encoding_of_scalar +")", 
                         line_prefix = "    ", max_len = 60, file=file)
    print ("  Outputs", file=file)
    tv_output_byte_array(Yb, test_vector_name = "Yb", 
                         line_prefix = "    ", max_len = 60, file=file)
    tv_output_byte_array(MSGb, test_vector_name = "MSGb", 
                         line_prefix = "    ", max_len = 60, file=file)
    
    print ("~~~", file=file)
    print ("\n###  Test vector for secret points K\n", file=file)
    print ("~~~", file=file)
    K1 = G.scalar_mult_vfy(ya,Yb)
    K2 = G.scalar_mult_vfy(yb,Ya)
    tv_output_byte_array(K1, test_vector_name = "scalar_mult_vfy(ya,Yb)", 
                         line_prefix = "    ", max_len = 60, file=file)
    tv_output_byte_array(K2, test_vector_name = "scalar_mult_vfy(yb,Ya)", 
                         line_prefix = "    ", max_len = 60, file=file)
    print ("~~~\n", file=file)
    if (K1 != K2):
        print ("Diffie-Hellman did fail!")
    K = K1
    
    ISK_IR = CPace_ISK(H,G.DSI_ISK,sid,K,MSGa,MSGb,doPrint = 1, symmetric_execution = False, file=file)
    ISK_SY = CPace_ISK(H,G.DSI_ISK,sid,K,MSGa,MSGb,doPrint = 1, symmetric_execution = True, file=file)
    
    if with_ANSI_C_initializers:
        print ("\n###  Corresponding ANSI-C initializers\n", file=file)
        print ("~~~", file=file)
        print (ByteArrayToCInitializer(PRS, "tc_PRS"), file=file)
        print (ByteArrayToCInitializer(CI, "tc_CI"), file=file)
        print (ByteArrayToCInitializer(sid, "tc_sid"), file=file)
        print (ByteArrayToCInitializer(g, "tc_g"), file=file)
        print (ByteArrayToCInitializer(ya, "tc_ya"), file=file)
        print (ByteArrayToCInitializer(ADa, "tc_ADa"), file=file)
        print (ByteArrayToCInitializer(Ya, "tc_Ya"), file=file)
        print (ByteArrayToCInitializer(yb, "tc_yb"), file=file)
        print (ByteArrayToCInitializer(ADb, "tc_ADb"), file=file)
        print (ByteArrayToCInitializer(Yb, "tc_Yb"), file=file)
        print (ByteArrayToCInitializer(K1, "tc_K"), file=file)
        print (ByteArrayToCInitializer(ISK_IR, "tc_ISK_IR"), file=file)
        print (ByteArrayToCInitializer(ISK_SY, "tc_ISK_SY"), file=file)
        print ("~~~\n", file=file)


H = H_SHA512()
G = G_X25519()

generate_test_vector(H,G, file=sys.stdout)


##  Test vector for CPace using group X25519 and hash SHA-512


###  Test vectors for calculate_generator with group X25519

~~~
  Inputs
    H   = SHA-512 with input block size 128 bytes.
    PRS = b'password' ; ZPAD length: 118 ; DSI = b'CPace255'
    CI = b'\nAinitiator\nBresponder'
    CI = 0a41696e69746961746f720a42726573706f6e646572
    sid = 7e4b4791d6a8ef019b936c79fb7f2c57
  Outputs
    hash generator string: (length: 32 bytes)
      5cb423cc3a5a9355bb90fceb67c97a7b5787df93faf4562789d705e3
      b2848d86
    after decoding to coordinate: (length: 32 bytes)
      5cb423cc3a5a9355bb90fceb67c97a7b5787df93faf4562789d705e3
      b2848d06
    generator g: (length: 32 bytes)
      2cddcc94b38d059a7b305bb0b8934b5b1ed45c5a5cb039f9cd00ab11
      ce92730d
~~~
<class 'bytes'> b'TH\xfd\x963sNp2\x10\xb6\x1d\\\xab\xb11\n(8(\x95\xd5mI\x05QCj\xb39\x86D'
<class 'bytes'> b'ADa'

###  Test vector for MSGa

~~~
  Inputs
    ADa = b'ADa'
    ya (little endian): (length: 32 bytes)
      232527dee2cfd

In [15]:
H = H_SHAKE256()
G = G_X448()
generate_test_vector(H,G)

##  Test vector for CPace using group X448 and hash SHAKE-256


###  Test vectors for calculate_generator with group X448

~~~
  Inputs
    H   = SHAKE-256 with input block size 136 bytes.
    PRS = b'password' ; ZPAD length: 126 ; DSI = b'CPace448'
    CI = b'\nAinitiator\nBresponder'
    CI = 0a41696e69746961746f720a42726573706f6e646572
    sid = 5223e0cdc45d6575668d64c552004124
  Outputs
    hash generator string: (length: 56 bytes)
      c1658ad06392f4eb5a23294d49210744aea89bf56cd9d1497b0b6ca0
      d4a9172fedd1e9d8376794c166ebbe05b598c051cbad24b03892e841
    after decoding to coordinate: (length: 32 bytes)
      c1658ad06392f4eb5a23294d49210744aea89bf56cd9d1497b0b6ca0
      d4a9172f
    generator g: (length: 56 bytes)
      402906591ba645f89b94dc93559c9c423a35d5eaf2878da0fd11b912
      aee50ffbf537a6b3bf72c28f3a12cf521eac520d2630806ee2b2f41d
~~~
<class 'bytes'> b"z$T\xa2\xff\xa1\x8c\t\xf8\xb5\xb6\n\xc9\x00\xf1\x9d-?\xb7\xb0\x1b\xb9\xcf\xe0}Z\xe9\x9d'\xbf\x89\x1a\xeb2\x1c5c\xa1\x7f

In [16]:
# Test vectors for the C Code, Elligator2

print (ByteArrayToCInitializer(IntegerToByteArray(etc1_in), "EllTestCase1_in"))
print (ByteArrayToCInitializer(IntegerToByteArray(etc1_out), "EllTestCase1_out"))

print (ByteArrayToCInitializer(IntegerToByteArray(etc2_in), "EllTestCase2_in"))
print (ByteArrayToCInitializer(IntegerToByteArray(etc2_out), "EllTestCase2_out"))


const uint8_t EllTestCase1_in[] = {
 0xbc,0x14,0x9a,0x46,0xd2,0x93,0xb0,0xae,0xea,0x34,0x58,0x13,
 0x49,0xd7,0x2f,0x8a,0x5a,0x96,0xcd,0x53,0x11,0x02,0xd6,0x73,
 0x79,0xcd,0x9b,0xfa,0xdd,0x4e,0xc8,0x00,
};
const uint8_t EllTestCase1_out[] = {
 0x66,0xb6,0x8f,0x75,0x75,0xcd,0x28,0x24,0x03,0xfc,0x2b,0xd3,
 0x23,0xff,0x04,0x60,0x12,0x03,0xc1,0xec,0x55,0x16,0xce,0x24,
 0x7f,0x7c,0x0a,0xdb,0xef,0x05,0xd3,0x67,
};
const uint8_t EllTestCase2_in[] = {
 0x89,0xcf,0x55,0xd4,0xb5,0xd3,0xf8,0x4b,0x16,0x34,0x95,0x7a,
 0xc5,0x03,0xa3,0x2b,0x84,0xba,0x11,0x47,0x1a,0x96,0xb2,0x27,
 0xbc,0xa7,0x0a,0x0c,0x3b,0xf2,0x63,0x75,
};
const uint8_t EllTestCase2_out[] = {
 0x1d,0xb1,0x63,0xc8,0x6c,0xec,0xa7,0x62,0x19,0x03,0xc9,0x41,
 0x2d,0x6d,0xc7,0x1b,0x4e,0xd2,0x63,0xb6,0x87,0xee,0xd0,0x92,
 0xb1,0x94,0xb5,0xe5,0x40,0xbb,0xa3,0x08,
};


In [ ]:
#

In [ ]:
#8.) Test vector output for ANSI-C, CPace




In [18]:
import binascii
import random
import hashlib
import sys

from sagelib.hash_to_field import I2OSP, OS2IP
from sagelib.suite_p256 import *
from sagelib.suite_p384 import *
from sagelib.suite_p521 import *

# Definitions for Short-Weierstrass-Curves

class G_ShortWeierstrass():
    def __init__(self, mapping_primitive):        
        self.map = mapping_primitive
        self.curve = self.map("some arbitrary string").curve();
        
        self.field = self.curve.base_field();
        self.q = self.field.order()
        self.field_size_bytes = ceil(log(float(self.q),2) / 8)
        self.p = self.curve.order();
        self.name = mapping_primitive.curve_name
        
        if not (self.p.is_prime()):
            raise ValueError ("Group order for Short-Weierstrass must be prime")

        self.I = I2OSP(0,1) # Represent the neutral element as a single byte string "\00"

        self.DSI = b"CPace" + mapping_primitive.suite_name.encode("ascii")
        self.DSI_ISK = self.DSI + b"_ISK"
        self.encoding_of_scalar = "big endian"
      
    def sample_scalar(self, deterministic_scalar_for_test_vectors = "False"):
        random_bytes_len =  self.field_size_bytes * 2
        if deterministic_scalar_for_test_vectors == "False":
            string = random_bytes(random_bytes_len)
        else:
            H = H_SHAKE256()
            string = H.hash(deterministic_scalar_for_test_vectors, random_bytes_len)
        scalar = ByteArrayToInteger(string, random_bytes_len)
        scalar = scalar % self.p
        return I2OSP(scalar,self.field_size_bytes)

    def point_to_octets(self,point):
        if point == (point * 0):
            return b"\00" # Neutral element.
    
        x,y = point.xy()
        return b"\04" + I2OSP(x,self.field_size_bytes) + I2OSP(y,self.field_size_bytes)

    def octets_to_point(self,octets):
        if (octets[0] == 0) and (len(octets) == 1):
            point = self.curve.gens()[0]
            return point * 0 # neutral element.
        
        if not octets[0] == 4:
            raise ValueError("Only uncompressed format supported.")
           
        if not (len(octets) == 1 + self.field_size_bytes * 2):
            raise ValueError("Wrong length of field")
        
        xstr = octets[1:(self.field_size_bytes+1)]
        ystr = octets[(self.field_size_bytes+1):]
        return self.curve(OS2IP(xstr),OS2IP(ystr))
        
    def scalar_mult(self,scalar_octets,point_octets):
        point = self.octets_to_point(point_octets)        
        scalar = OS2IP(scalar_octets)
        return self.point_to_octets(point * scalar)

    def scalar_mult_vfy(self,scalar_octets,point_octets):
        scalar = OS2IP(scalar_octets)
        try:
            point = self.octets_to_point(point_octets)
        except:
            # Incorrect format or point is not on curve
            return self.I
        
        result_point = point * scalar
               
        if result_point == point * 0:
            return self.I
        else:
            x,y = result_point.xy()
            return I2OSP(x,self.field_size_bytes)
        
    def calculate_generator(self, H, PRS, CI, sid, print_test_vector_info = False, file = sys.stdout ):
        (gen_string, len_zpad) = generator_string(PRS, G.DSI,CI,sid,H.s_in_bytes)
        result = self.map(gen_string)
        if print_test_vector_info:
            print ("\n###  Test vectors for calculate_generator with group "+self.name+"\n", file =file)
            print ("~~~", file = file)
            print ("  Inputs", file = file)
            print ("    H   =", H.name, "with input block size", H.s_in_bytes, "bytes.", file = file)
            print ("    PRS =", PRS, "; ZPAD length:", len_zpad,";",file = file);
            print ("    DSI =", self.DSI, file = file)
            print ("    CI =", CI, file = file)
            print ("    CI =", ByteArrayToLEPrintString(CI), file = file)
            print ("    sid =", ByteArrayToLEPrintString(sid), file = file)
            print ("  Outputs", file = file)
            tv_output_byte_array(gen_string, test_vector_name = "string passed to map", 
                                 line_prefix = "    ", max_len = 60, file = file)
            tv_output_byte_array(self.point_to_octets(result), test_vector_name = "generator g", 
                                 line_prefix = "    ", max_len = 60,file = file)
            print ("~~~", file = file)
        return self.point_to_octets(result)

prime = 2^255 - 19
F = GF(prime)
A = F(486662)
E = EllipticCurve(F, [0, A , 0, 1 , 0])
point = E.gens()[0]

#print (point_to_octets (point))

#print (point)
#print (octets_to_point(point_to_octets (point),E))

#print (p256_sswu_nu("Hallo.2"))

G_P256 = G_ShortWeierstrass(p256_sswu_nu)


def ByteArrayToInteger(k,numBytes=32):
    try:
        k_list = [ord(b) for b in k]
    except:
        k_list = [b for b in k]
 
    if numBytes < len(k_list):
        numBytes = len(k_list)

    return sum((k_list[i] << (8 * i)) for i in range(numBytes))


scalar = G_P256.sample_scalar()
#scalar = b"\0"
generator = G_P256.calculate_generator(H_SHA256(),b"PRS", b"ci",b"sid", False)
Y = G_P256.scalar_mult(scalar, generator)

#print (G_P256.octets_to_point(Y))

K = G_P256.scalar_mult_vfy(scalar, Y)

print (scalar)
print (generator)
print (Y)
print (K)

ModuleNotFoundError: No module named 'sagelib.hash_to_field'

In [ ]:
H = H_SHA256()
G = G_ShortWeierstrass(p256_sswu_nu)
#generate_test_vector(H,G)

H = H_SHA384()
G = G_ShortWeierstrass(p384_sswu_nu)
generate_test_vector(H,G)

H = H_SHA512()
G = G_ShortWeierstrass(p521_sswu_nu)
#generate_test_vector(H,G)



In [ ]:
import binascii
import random
import hashlib
import sys

from ristretto_decaf import *

# Definitions for Ristretto and Decaf

class G_CoffeeEcosystem():
    def __init__(self, coffee_point_class):
        self.point_class = coffee_point_class
        self.field = coffee_point_class.F
        self.q = coffee_point_class.P
        self.field_size_bytes = ceil(log(self.q,2) / 8)
        self.p = coffee_point_class.order
        self.name = coffee_point_class.name
        
        self.I = (coffee_point_class.map(H_SHAKE256().hash(b"1234",self.field_size_bytes * 2)) * 0).encode()
        self.DSI = b"CPace" + self.name.encode("ascii")
        self.DSI_ISK = self.DSI + b"_ISK"
        self.encoding_of_scalar = "little endian"

    def sample_scalar(self, deterministic_scalar_for_test_vectors = "False"):
        if deterministic_scalar_for_test_vectors == "False":
            value = random_bytes(self.field_size_bytes)
        else:
            value = H_SHAKE256().hash(deterministic_scalar_for_test_vectors)
        
        value_int = ByteArrayToInteger(value, self.field_size_bytes)
        reduced_value = value_int % self.p
        result = IntegerToByteArray(reduced_value,self.field_size_bytes)        
        return result

    def scalar_mult(self,scalar,encoded_point):
        point = self.point_class.decode(encoded_point)
        scalar_as_int = ByteArrayToInteger(scalar, self.field_size_bytes);
        return (point * scalar_as_int).encode()

    def scalar_mult_vfy(self,scalar,encoded_point):
        scalar_as_int = ByteArrayToInteger(scalar, self.field_size_bytes);
        try:
            point = self.point_class.decode(encoded_point);
        except:
            # Decoding of point failed.
            return self.I
        return (point * scalar_as_int).encode()

    def calculate_generator(self, H, PRS, CI, sid, print_test_vector_info = False, file = sys.stdout):
        (gen_string, len_zpad) = generator_string(PRS, G.DSI,CI,sid,H.s_in_bytes)
        string_hash = H.hash(gen_string, self.field_size_bytes * 2)
        result = self.point_class.map(string_hash)
        if print_test_vector_info:
            print ("\n###  Test vectors for calculate_generator with group "+self.name+"\n", file = file)
            print ("~~~", file = file)
            print ("  Inputs", file = file)
            print ("    H   =", H.name, "with input block size", H.s_in_bytes, "bytes.", file = file)
            print ("    PRS =", PRS, "; ZPAD length:", len_zpad,"; DSI =", self.DSI, file = file)
            print ("    CI =", CI, file = file)
            print ("    CI =", ByteArrayToLEPrintString(CI), file = file)
            print ("    sid =", ByteArrayToLEPrintString(sid), file = file)
            print ("  Outputs", file = file)
            tv_output_byte_array(gen_string, test_vector_name = "hash generator string", 
                                 line_prefix = "    ", max_len = 60, file = file)
            tv_output_byte_array(string_hash, test_vector_name = "hash result", 
                                 line_prefix = "    ", max_len = 60, file = file)
            tv_output_byte_array(result.encode(), test_vector_name = "encoded generator g", 
                                 line_prefix = "    ", max_len = 60, file = file)
            print ("~~~\n", file = file)
        return result.encode()

H = H_SHA512()
G = G_CoffeeEcosystem(Ed25519Point)

H = H_SHAKE256()
G = G_CoffeeEcosystem(Ed448GoldilocksPoint)
generate_test_vector(H,G)


P255 = Ed25519Point()

P448 = Ed448GoldilocksPoint()

(P255.map(H_SHA512().hash(b"1234")) * 3).encode()

G.I

In [ ]:

with open('testvectors.txt', 'w') as f:

    generate_testvectors_string_functions(file = f)
    
    print("# Test vectors", file = f)
    
    output_test_vectors_for_weak_points_255(file = f)
    H = H_SHA512()
    G = G_X25519()
    generate_test_vector(H,G, file=f)
 
    output_test_vectors_for_weak_points_448(file = f)
    H = H_SHAKE256()
    G = G_X448()
    generate_test_vector(H,G, file=f)
   
    H = H_SHA512()
    G = G_CoffeeEcosystem(Ed25519Point)
    generate_test_vector(H,G, file=f)

    H = H_SHAKE256()
    G = G_CoffeeEcosystem(Ed448GoldilocksPoint)
    generate_test_vector(H,G, file=f)
    
    H = H_SHA256()
    G = G_ShortWeierstrass(p256_sswu_nu)
    generate_test_vector(H,G, file=f)

    H = H_SHA384()
    G = G_ShortWeierstrass(p384_sswu_nu)
    generate_test_vector(H,G, file=f)

    H = H_SHA512()
    G = G_ShortWeierstrass(p521_sswu_nu)
    generate_test_vector(H,G, file=f)
    

    

In [ ]:
H = H_SHA512()
G = G_X25519()
generate_test_vector(H,G)


In [ ]:
values = H_SHA256().hash(b"1234")
print (values,"\n")
print (ByteArrayToLEPrintString(values),"\n")
print (ByteArrayToCInitializer(values,"values"),"\n")

In [ ]:
print (w)

In [ ]:
IntegerToLEPrintString(0x232527dee2cfde76fb425b6d88818630eea7ea263fac28d89f52d096c563b1e)